# Урок 6
Взаимосвязь величин. Показатели корреляции. Корреляционный анализ. Проверка на нормальность

__Домашнее задание__

In [24]:
from scipy import stats
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from itertools import combinations

__Задача 1__

Даны значения величины заработной платы заемщиков банка (salary) и значения их поведенческого кредитного скоринга (scoring):

salary = [35, 45, 190, 200, 40, 70, 54, 150, 120, 110]
scoring = [401, 574, 874, 919, 459, 739, 653, 902, 746, 832]

Используя только встроенные питоновские функции и структуры данных (т.е. без библиотек numpy, pandas и др.) найдите:
1. ковариацию (смещённую и несмещённую) этих двух величин,
2. коэффициент корреляции Пирсона.

Можно затем посчитать те же значения с использованием библиотек, чтобы проверить себя.

1.

Смещённая ковариация
$$
cov_{biased} = \dfrac{1}{n} \displaystyle\sum_{i = 1}^n \left( x_i - \overline{x} \right) \cdot \left( y_i - \overline{y} \right)
$$

In [26]:
#N

x = [35, 45, 190, 200, 40, 70, 54, 150, 120, 110]
y = [401, 574, 874, 919, 459, 739, 653, 902, 746, 832]

n = len(x)


x_mean = sum(x) / n
y_mean = sum(y) / n

E = 0

for i in range(n):
    E += (x[i] - x_mean) * (y[i] - y_mean)
    
cov_biased = E / n
print(f'Смещённая ковариация {cov_biased}')

Смещённая ковариация 9157.84


Проверяем Смещённую ковариацию в библиотеке numpy

In [27]:
np.cov(x, y, ddof=0)

array([[ 3494.64,  9157.84],
       [ 9157.84, 30468.89]])

Несмещённая ковариация $$
cov = \dfrac{1}{n-1} \displaystyle\sum_{i = 1}^n \left( x_i - \overline{x} \right) \cdot \left( y_i - \overline{y} \right)
$$

In [31]:
# N-1 

cov_biased = E / (n-1)
print(f'Смещённая ковариация {round(cov_biased,2)}')

Смещённая ковариация 10175.38


Проверяем несмещённую ковариацию в библиотеке numpy

In [34]:
np.cov(x, y, ddof=1)

array([[ 3882.93333333, 10175.37777778],
       [10175.37777778, 33854.32222222]])

2. коэффициент корреляции Пирсона. $$r_{XY} = \dfrac{\sigma_{XY}}{\sigma_X \cdot \sigma_Y}$$

In [46]:
# covariance unbiased

cov = np.cov(x, y, ddof=1)[1,0]

# Variances unbiased

VarX = sum((x_i - x_mean)**2 for x_i in x) / (n - 1)
VarY = sum((y_i - y_mean)**2 for y_i in y) / (n - 1)

# Standard Deviations unbiased

std_X = VarX ** 0.5
std_Y = VarY ** 0.5

Pir_Corr = cov / (std_X * std_Y)
Pir_Corr

0.887490092073916

Проверяем коэффициент корреляции Пирсона через numpy

In [49]:
np.corrcoef(x,y)

array([[1.        , 0.88749009],
       [0.88749009, 1.        ]])

__Задача 2__

Проведите тест на значимость коэффициента корреляции Пирсона, найденного в предыдущей задаче. Что для этого нужно знать:
* Нулевая гипотеза: реальный коэффициент корреляции равен 0. Альтернативная гипотеза двухсторонняя.
* Статистика: t = r * sqrt(n - 2) / sqrt(1 - r ** 2), где r - коэффициент корреляции Пирсона, посчитанный по выборке.
* В предположении верности нулевой гипотезы эта статистика имеет распределение Стьюдента с параметром df = n - 2.

Подсчитываем критические области

In [57]:
# альфа 0.5 и два хвоста 

alpha = 0.05
ne = len(x)

t1 = stats.t.ppf(alpha/2, df=n-2)
t2 = stats.t.ppf(1 - alpha/2, df=n-2)

print(f'критичиская область {t1}, {t2}')

критичиская область -2.306004135033371, 2.3060041350333704


$$ t = \dfrac{r \sqrt{n-2}}{\sqrt{1 - r^2} }$$

Считаем  Т-статистику по следущей формуле так как в выборке мене 30 элементов 

In [71]:
t = r * np.sqrt(n-2) / np.sqrt(1-r**2)
print(f't : {round(t,2)} \nt Попала в критичскую область, значит отвергаем нулевую гипотезу !!!')

t : 5.45 
t Попала в критичскую область, значит отвергаем нулевую гипотезу !!!


__Задача 3__

Измерены значения IQ выборки студентов, обучающихся в местных технических вузах:

131, 125, 115, 122, 131, 115, 107, 99, 125, 111

Известно, что в генеральной совокупности IQ распределен нормально. Найдите доверительный интервал для математического ожидания с надежностью 0.95.

так как __дисперсия неизвестна__ мы будем использовать статистику
$$t = \dfrac{\overline{X} - \mu}{\sigma_X / \sqrt{n}},$$
где $\sigma_X$ — выборочное среднее квадратическое отклонение. Эта статистика имеет распределение Стьюдента, поэтому
$$P \left( t_{\alpha / 2, \: n - 1} \leq t \leq t_{1 - \alpha / 2, \: n - 1} \right) = p,$$
где $t_{\beta, \: n - 1}$ — квантиль порядка $\beta$ для распределения Стьюдента с параметром $df = n - 1$. Аналогичным способом получаем доверительный интервал:
$$P \left( \overline{X} + t_{\alpha / 2, \: n - 1} \cdot \dfrac{\sigma_X}{\sqrt{n}} \leq \mu \leq \overline{X} + t_{1 - \alpha / 2, \: n - 1} \cdot \dfrac{\sigma_X}{\sqrt{n}} \right) = p$$

In [78]:
IQ = np.array([131, 125, 115, 122, 131, 115, 107, 99, 125, 111])

n    = IQ.shape[0]
mean = IQ.mean()
std  = IQ.std(ddof=1)

n, mean, std

(10, 118.1, 10.54566788359614)

Подсчитаем квантили

In [85]:
p = 0.95
alpha = 1 - p

t1 = stats.t.ppf(alpha/2, df=n -1)
t2 = stats.t.ppf(1-alpha/2, df=n-1)

t1,t2

(-2.2621571627409915, 2.2621571627409915)

Доверительный Интервал

In [88]:
(mean + t1 * std / np.sqrt(n), mean + t2 *std / np.sqrt(n))

(110.55608365158724, 125.64391634841274)